In [5]:
import sys

# 设置 sys.argv 为默认值或仅包含脚本名称
sys.argv = ['']

# 然后你可以安全地调用你的解析代码
# args = parser.parse_args()


In [6]:
import time
from options.my_train_options import TrainOptions
from data import create_dataset
from models import create_model
from util.visualizer import Visualizer




In [7]:
opt = TrainOptions().parse()   # get training options


----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: True                          
                crop_size: 256                           
                 dataroot: ./datasets/my_aligned_dataset 
             dataset_mode: aligned                       
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: -1                            
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: latest                        
              epoch_count: 6  

In [8]:
dataset = create_dataset(opt)  # create a dataset given opt.dataset_mode and other options


dataset [AlignedDataset] was created


In [9]:
dataset_size = len(dataset)    # get the number of images in the dataset.
print('The number of training images = %d' % dataset_size)


The number of training images = 3879


In [10]:

model = create_model(opt)      # create a model given opt.model and other options
model.setup(opt)               # regular setup: load and print networks; create schedulers


initialize network with normal
initialize network with normal
model [Pix2PixModel] was created
loading the model from ./checkpoints\facades_pix2pix\latest_net_G.pth
loading the model from ./checkpoints\facades_pix2pix\latest_net_D.pth
---------- Networks initialized -------------
[Network G] Total number of parameters : 54.414 M
[Network D] Total number of parameters : 2.769 M
-----------------------------------------------


In [11]:
model

In [12]:
visualizer = Visualizer(opt)   # create a visualizer that display/save images and plots
total_iters = 0                # the total number of training iterations


create web directory ./checkpoints\facades_pix2pix\web...


In [13]:

# outer loop for different epochs; we save the model by <epoch_count>, <epoch_count>+<save_latest_freq>
epoch_start_time = time.time()  # timer for entire epoch
iter_data_time = time.time()    # timer for data loading per iteration
epoch_iter = 0                  # the number of training iterations in current epoch, reset to 0 every epoch


In [14]:
visualizer.reset()              # reset the visualizer: make sure it saves the results to HTML at least once every epoch


In [15]:
model.update_learning_rate()    # update learning rates in the beginning of every epoch.


learning rate 0.0002000 -> 0.0002000


In [16]:
epoch = 0



In [17]:
type(dataset)

data.CustomDatasetDataLoader

In [11]:
# 假设 'dataset' 是一个 PyTorch DataLoader 实例
data_iterator = iter(dataset)

# 使用 next() 方法获取一批数据
one_batch = next(data_iterator)

one_batch

{'A': tensor([[[[-0.6078, -0.6157, -0.6235,  ..., -0.7882, -0.7961, -0.7961],
           [-0.6078, -0.6157, -0.6235,  ..., -0.7882, -0.7961, -0.7961],
           [-0.6000, -0.6078, -0.6157,  ..., -0.7725, -0.7804, -0.7804],
           ...,
           [-0.5608, -0.5294, -0.5216,  ..., -0.6706, -0.6235, -0.6314],
           [-0.5451, -0.5137, -0.5059,  ..., -0.6706, -0.6235, -0.6314],
           [-0.5373, -0.4980, -0.4902,  ..., -0.6784, -0.6235, -0.6392]],
 
          [[ 0.2471,  0.2392,  0.2314,  ...,  0.4039,  0.3961,  0.3961],
           [ 0.2471,  0.2392,  0.2314,  ...,  0.4039,  0.3961,  0.3961],
           [ 0.2392,  0.2314,  0.2235,  ...,  0.4039,  0.3961,  0.3961],
           ...,
           [-0.1373, -0.1216, -0.1216,  ...,  0.0510,  0.1059,  0.1137],
           [-0.1373, -0.1059, -0.1059,  ...,  0.0510,  0.0980,  0.1137],
           [-0.1294, -0.0902, -0.0980,  ...,  0.0431,  0.0902,  0.1059]],
 
          [[ 0.0039, -0.0039, -0.0118,  ...,  0.2235,  0.2157,  0.2157],
        

In [ ]:
# 提取输入图像（A）和目标图像（B）的张量
A_images = one_batch['A']
B_images = one_batch['B']

# 如果存在，提取图像路径（A_paths 和 B_paths）
A_paths = one_batch.get('A_paths', None)  # 使用 get 方法可以避免 KeyError，如果键不存在，返回 None
B_paths = one_batch.get('B_paths', None)

# 现在你可以使用 A_images 和 B_images 进行模型训练或测试
# A_paths 和 B_paths 可以用于追踪图像的来源或进行其他的记录工作


In [ ]:
A_images.shape

In [ ]:
model.set_input(one_batch)

In [ ]:

# for i, data in enumerate(dataset):  # inner loop within one epoch
#     iter_start_time = time.time()  # timer for computation per iteration
#     if total_iters % opt.print_freq == 0:
#         t_data = iter_start_time - iter_data_time

#     total_iters += opt.batch_size
#     epoch_iter += opt.batch_size
#     model.set_input(data)

In [ ]:
     # unpack data from dataset and apply preprocessing
model.optimize_parameters()   # calculate loss functions, get gradients, update network weights


In [ ]:

if total_iters % opt.display_freq == 0:   # display images on visdom and save images to a HTML file
    save_result = total_iters % opt.update_html_freq == 0
    model.compute_visuals()
    visualizer.display_current_results(model.get_current_visuals(), epoch, save_result)

if total_iters % opt.print_freq == 0:    # print training losses and save logging information to the disk
    losses = model.get_current_losses()
    t_comp = (time.time() - iter_start_time) / opt.batch_size
    visualizer.print_current_losses(epoch, epoch_iter, losses, t_comp, t_data)
    if opt.display_id > 0:
        visualizer.plot_current_losses(epoch, float(epoch_iter) / dataset_size, losses)

if total_iters % opt.save_latest_freq == 0:   # cache our latest model every <save_latest_freq> iterations
    print('saving the latest model (epoch %d, total_iters %d)' % (epoch, total_iters))
    save_suffix = 'iter_%d' % total_iters if opt.save_by_iter else 'latest'
    model.save_networks(save_suffix)

iter_data_time = time.time()

# if epoch % opt.save_epoch_freq == 0:              # cache our model every <save_epoch_freq> epochs
#     print('saving the model at the end of epoch %d, iters %d' % (epoch, total_iters))
#     model.save_networks('latest')
#     model.save_networks(epoch)

# print('End of epoch %d / %d \t Time Taken: %d sec' % (epoch, opt.n_epochs + opt.n_epochs_decay, time.time() - epoch_start_time))


In [ ]:

for i, data in enumerate(dataset):  # inner loop within one epoch
    iter_start_time = time.time()  # timer for computation per iteration
    if total_iters % opt.print_freq == 0:
        t_data = iter_start_time - iter_data_time

    total_iters += opt.batch_size
    epoch_iter += opt.batch_size
    model.set_input(data)         # unpack data from dataset and apply preprocessing
    model.optimize_parameters()   # calculate loss functions, get gradients, update network weights

    if total_iters % opt.display_freq == 0:   # display images on visdom and save images to a HTML file
        save_result = total_iters % opt.update_html_freq == 0
        model.compute_visuals()
        visualizer.display_current_results(model.get_current_visuals(), epoch, save_result)

    if total_iters % opt.print_freq == 0:    # print training losses and save logging information to the disk
        losses = model.get_current_losses()
        t_comp = (time.time() - iter_start_time) / opt.batch_size
        visualizer.print_current_losses(epoch, epoch_iter, losses, t_comp, t_data)
        if opt.display_id > 0:
            visualizer.plot_current_losses(epoch, float(epoch_iter) / dataset_size, losses)

    if total_iters % opt.save_latest_freq == 0:   # cache our latest model every <save_latest_freq> iterations
        print('saving the latest model (epoch %d, total_iters %d)' % (epoch, total_iters))
        save_suffix = 'iter_%d' % total_iters if opt.save_by_iter else 'latest'
        model.save_networks(save_suffix)

    iter_data_time = time.time()
if epoch % opt.save_epoch_freq == 0:              # cache our model every <save_epoch_freq> epochs
    print('saving the model at the end of epoch %d, iters %d' % (epoch, total_iters))
    model.save_networks('latest')
    model.save_networks(epoch)

print('End of epoch %d / %d \t Time Taken: %d sec' % (epoch, opt.n_epochs + opt.n_epochs_decay, time.time() - epoch_start_time))


In [ ]:

# for epoch in range(opt.epoch_count, opt.n_epochs + opt.n_epochs_decay + 1):    # outer loop for different epochs; we save the model by <epoch_count>, <epoch_count>+<save_latest_freq>
#     epoch_start_time = time.time()  # timer for entire epoch
#     iter_data_time = time.time()    # timer for data loading per iteration
#     epoch_iter = 0                  # the number of training iterations in current epoch, reset to 0 every epoch
#     visualizer.reset()              # reset the visualizer: make sure it saves the results to HTML at least once every epoch
#     model.update_learning_rate()    # update learning rates in the beginning of every epoch.
#     for i, data in enumerate(dataset):  # inner loop within one epoch
#         iter_start_time = time.time()  # timer for computation per iteration
#         if total_iters % opt.print_freq == 0:
#             t_data = iter_start_time - iter_data_time

#         total_iters += opt.batch_size
#         epoch_iter += opt.batch_size
#         model.set_input(data)         # unpack data from dataset and apply preprocessing
#         model.optimize_parameters()   # calculate loss functions, get gradients, update network weights

#         if total_iters % opt.display_freq == 0:   # display images on visdom and save images to a HTML file
#             save_result = total_iters % opt.update_html_freq == 0
#             model.compute_visuals()
#             visualizer.display_current_results(model.get_current_visuals(), epoch, save_result)

#         if total_iters % opt.print_freq == 0:    # print training losses and save logging information to the disk
#             losses = model.get_current_losses()
#             t_comp = (time.time() - iter_start_time) / opt.batch_size
#             visualizer.print_current_losses(epoch, epoch_iter, losses, t_comp, t_data)
#             if opt.display_id > 0:
#                 visualizer.plot_current_losses(epoch, float(epoch_iter) / dataset_size, losses)

#         if total_iters % opt.save_latest_freq == 0:   # cache our latest model every <save_latest_freq> iterations
#             print('saving the latest model (epoch %d, total_iters %d)' % (epoch, total_iters))
#             save_suffix = 'iter_%d' % total_iters if opt.save_by_iter else 'latest'
#             model.save_networks(save_suffix)

#         iter_data_time = time.time()
#     if epoch % opt.save_epoch_freq == 0:              # cache our model every <save_epoch_freq> epochs
#         print('saving the model at the end of epoch %d, iters %d' % (epoch, total_iters))
#         model.save_networks('latest')
#         model.save_networks(epoch)

#     print('End of epoch %d / %d \t Time Taken: %d sec' % (epoch, opt.n_epochs + opt.n_epochs_decay, time.time() - epoch_start_time))


In [1]:
"""General-purpose training script for image-to-image translation.

This script works for various models (with option '--model': e.g., pix2pix, cyclegan, colorization) and
different datasets (with option '--dataset_mode': e.g., aligned, unaligned, single, colorization).
You need to specify the dataset ('--dataroot'), experiment name ('--name'), and model ('--model').

It first creates model, dataset, and visualizer given the option.
It then does standard network training. During the training, it also visualize/save the images, print/save the loss plot, and save models.
The script supports continue/resume training. Use '--continue_train' to resume your previous training.

Example:
    Train a CycleGAN model:
        python train.py --dataroot ./datasets/maps --name maps_cyclegan --model cycle_gan
    Train a pix2pix model:
        python train.py --dataroot ./datasets/facades --name facades_pix2pix --model pix2pix --direction BtoA

See options/base_options.py and options/train_options.py for more training options.
See training and test tips at: https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/blob/master/docs/tips.md
See frequently asked questions at: https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/blob/master/docs/qa.md
"""
import time
from options.my_train_options import TrainOptions
from data import create_dataset
from models import create_model
from util.visualizer import Visualizer


In [2]:
import sys

# 设置 sys.argv 为默认值或仅包含脚本名称
sys.argv = ['']

# 然后你可以安全地调用你的解析代码
# args = parser.parse_args()


In [3]:
opt = TrainOptions().parse()   # get training options


----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: True                          
                crop_size: 256                           
                 dataroot: ./datasets/my_aligned_dataset 
             dataset_mode: aligned                       
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: -1                            
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: latest                        
              epoch_count: 6  

In [4]:

opt = TrainOptions().parse()   # get training options
dataset = create_dataset(opt)  # create a dataset given opt.dataset_mode and other options
dataset_size = len(dataset)    # get the number of images in the dataset.
print('The number of training images = %d' % dataset_size)

model = create_model(opt)      # create a model given opt.model and other options
model.setup(opt)               # regular setup: load and print networks; create schedulers
visualizer = Visualizer(opt)   # create a visualizer that display/save images and plots
total_iters = 0                # the total number of training iterations

for epoch in range(opt.epoch_count, opt.n_epochs + opt.n_epochs_decay + 1):    # outer loop for different epochs; we save the model by <epoch_count>, <epoch_count>+<save_latest_freq>
    epoch_start_time = time.time()  # timer for entire epoch
    iter_data_time = time.time()    # timer for data loading per iteration
    epoch_iter = 0                  # the number of training iterations in current epoch, reset to 0 every epoch
    visualizer.reset()              # reset the visualizer: make sure it saves the results to HTML at least once every epoch
    model.update_learning_rate()    # update learning rates in the beginning of every epoch.
    for i, data in enumerate(dataset):  # inner loop within one epoch
        iter_start_time = time.time()  # timer for computation per iteration
        if total_iters % opt.print_freq == 0:
            t_data = iter_start_time - iter_data_time

        total_iters += opt.batch_size
        epoch_iter += opt.batch_size
        model.set_input(data)         # unpack data from dataset and apply preprocessing
        model.optimize_parameters()   # calculate loss functions, get gradients, update network weights

        if total_iters % opt.display_freq == 0:   # display images on visdom and save images to a HTML file
            save_result = total_iters % opt.update_html_freq == 0
            model.compute_visuals()
            visualizer.display_current_results(model.get_current_visuals(), epoch, save_result)

        if total_iters % opt.print_freq == 0:    # print training losses and save logging information to the disk
            losses = model.get_current_losses()
            t_comp = (time.time() - iter_start_time) / opt.batch_size
            visualizer.print_current_losses(epoch, epoch_iter, losses, t_comp, t_data)
            if opt.display_id > 0:
                visualizer.plot_current_losses(epoch, float(epoch_iter) / dataset_size, losses)

        if total_iters % opt.save_latest_freq == 0:   # cache our latest model every <save_latest_freq> iterations
            print('saving the latest model (epoch %d, total_iters %d)' % (epoch, total_iters))
            save_suffix = 'iter_%d' % total_iters if opt.save_by_iter else 'latest'
            model.save_networks(save_suffix)

        iter_data_time = time.time()
    if epoch % opt.save_epoch_freq == 0:              # cache our model every <save_epoch_freq> epochs
        print('saving the model at the end of epoch %d, iters %d' % (epoch, total_iters))
        model.save_networks('latest')
        model.save_networks(epoch)

    print('End of epoch %d / %d \t Time Taken: %d sec' % (epoch, opt.n_epochs + opt.n_epochs_decay, time.time() - epoch_start_time))


----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: True                          
                crop_size: 256                           
                 dataroot: ./datasets/my_aligned_dataset 
             dataset_mode: aligned                       
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: -1                            
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: latest                        
              epoch_count: 6  

e:\anaconda3\envs\inpainting-basic\Lib\site-packages\torch\optim\lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


(epoch: 6, iters: 100, time: 0.079, data: 8.459) G_GAN: 1.647 G_L1: 10.046 D_real: 2.255 D_fake: 0.091 
(epoch: 6, iters: 200, time: 0.079, data: 0.000) G_GAN: 1.002 G_L1: 10.835 D_real: 0.761 D_fake: 0.414 
(epoch: 6, iters: 300, time: 0.078, data: 0.001) G_GAN: 1.435 G_L1: 9.157 D_real: 0.823 D_fake: 0.333 
(epoch: 6, iters: 400, time: 0.165, data: 0.000) G_GAN: 0.970 G_L1: 5.668 D_real: 0.365 D_fake: 0.656 
(epoch: 6, iters: 500, time: 0.081, data: 0.001) G_GAN: 1.155 G_L1: 8.075 D_real: 0.558 D_fake: 0.453 
(epoch: 6, iters: 600, time: 0.081, data: 0.000) G_GAN: 2.560 G_L1: 7.756 D_real: 0.115 D_fake: 0.421 
(epoch: 6, iters: 700, time: 0.083, data: 0.000) G_GAN: 1.432 G_L1: 8.092 D_real: 0.549 D_fake: 0.342 
(epoch: 6, iters: 800, time: 0.081, data: 0.000) G_GAN: 2.974 G_L1: 6.303 D_real: 0.466 D_fake: 1.366 
(epoch: 6, iters: 900, time: 0.080, data: 0.000) G_GAN: 1.076 G_L1: 7.239 D_real: 0.506 D_fake: 0.154 
(epoch: 6, iters: 1000, time: 0.081, data: 0.001) G_GAN: 2.671 G_L1: 10